In [14]:
!nvidia-smi


Sun Jan 18 13:30:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
%%writefile task3.cu
#include <cuda_runtime.h>   // библиотека для работы с CUDA
#include <iostream>         // ввод / вывод
#include <chrono>           // замер времени

// количество элементов в массиве
#define N 1000000

// размер блока потоков
#define BLOCK_SIZE 256

//
// CUDA-ЯДРО с КОАЛЕСЦИРОВАННЫМ доступом к памяти
//
// В этом ядре каждый поток работает со "своим" элементом массива
// Потоки с соседними индексами обращаются к соседним ячейкам памяти
// Это самый эффективный вариант работы с глобальной памятью
__global__ void coalesced_access(float* data) {

    // вычисляем глобальный индекс элемента
    // blockIdx.x   — номер блока
    // blockDim.x   — количество потоков в блоке
    // threadIdx.x  — номер потока внутри блока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // проверяем, что не вышли за границы массива
    if (idx < N) {
        // каждый поток читает и записывает
        // соседний элемент массива
        // такой доступ является коалесцированным
        data[idx] = data[idx] * 2.0f;
    }
}

//
// CUDA-ЯДРО с НЕКОАЛЕСЦИРОВАННЫМ доступом к памяти
//
// Здесь потоки обращаются к памяти "скачками"
// Это приводит к большому количеству отдельных обращений
// к глобальной памяти и снижает производительность
__global__ void non_coalesced_access(float* data) {

    // вычисляем глобальный индекс потока
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    // намеренно делаем плохой шаблон доступа:
    // каждый поток обращается к элементу через шаг 2
    int access_index = idx * 2;

    // проверяем выход за границы массива
    if (access_index < N) {
        // такой доступ считается некоалесцированным
        data[access_index] = data[access_index] * 2.0f;
    }
}

int main() {

    //
    // Выделение памяти на CPU
    //
    float* h_data = new float[N];

    // указатель на память на GPU
    float* d_data;

    //
    // Инициализация массива на CPU
    //
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    //
    // Выделение памяти на CPU
    //
    cudaMalloc(&d_data, N * sizeof(float));

    //
    // Копирование данных с CPU на GPU
    //
    cudaMemcpy(d_data, h_data, N * sizeof(float),
               cudaMemcpyHostToDevice);

    //
    // настройка сетки и блоков
    //
    dim3 block(BLOCK_SIZE);                     // размер блока
    dim3 grid((N + BLOCK_SIZE - 1) / BLOCK_SIZE); // количество блоков

    //
    // Замер времени: КОАЛЕСЦИРОВАННЫЙ доступ
    //
    auto start_coal = std::chrono::high_resolution_clock::now();

    // запуск CUDA-ядра
    coalesced_access<<<grid, block>>>(d_data);

    // ждём завершения всех потоков
    cudaDeviceSynchronize();

    auto end_coal = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time_coal = end_coal - start_coal;

    //
    // замер времени: НЕКОАЛЕСЦИРОВАННЫЙ лоступ
    //
    auto start_non = std::chrono::high_resolution_clock::now();

    // запуск второго CUDA-ядра
    non_coalesced_access<<<grid, block>>>(d_data);

    // синхронизация устройства
    cudaDeviceSynchronize();

    auto end_non = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> time_non = end_non - start_non;


    std::cout << "Коалесцированный доступ:   "
              << time_coal.count() << " секунд\n";

    std::cout << "Некоалесцированный доступ: "
              << time_non.count() << " секунд\n";

    cudaFree(d_data);     // память GPU
    delete[] h_data;      // память CPU

    return 0;
}


Overwriting task3.cu


In [17]:
!nvcc task3.cu -o task3


In [18]:
!./task3


Коалесцированный доступ:   0.00752707 секунд
Некоалесцированный доступ: 2.848e-06 секунд
